In [7]:
import Fitting_module
import Fitting_curves
import numpy

In [2]:
def SOPTD(parameters,u,tspan,yo):    # form is k/(t1s+1)(t2s+1)

    k,tau,zeta,theta = parameters

    G1 = lti([k], [tau**2,2*zeta*tau, 1])
    dt = tspan[1] - tspan[0]
    results = numpy.zeros(len(tspan))
    yvals = numpy.zeros(len(tspan))
    y = yo
    x = numpy.zeros([G1.A.shape[0], 1])

    for i, t in enumerate(tspan):
        yvals[i] = y
        t_interp = t - theta
        ydelayed = numpy.interp(t_interp, tspan, yvals)
        results[i] = ydelayed
        dx = G1.A.dot(x) + G1.B.dot(u)
        x += dx * dt
        y_prime = G1.C.dot(x) + G1.D.dot(u)
        y = y_prime[0][0] + yo

    return numpy.array(results)


In [25]:
def error_func(parameters,model,u,t,yo,simdata):

    if model == 'FOPTD':
        results = FOPTD(parameters,u,t,yo)
    elif model == 'SOPTD':
        results = SOPTD(parameters, u, t, yo)
    elif model == 'SOZPTD':
        results = SOZPTD(parameters,u,t,yo)

    abs_error = abs(simdata - results)
    err = sum(abs_error)
    return err

In [26]:
def run_all_fits(names,fit_types,initials,yo_vals,u_vals,data):
    print('Initializing')
    fitted_params = []
    reserror = numpy.zeros(len(names))
    print('Running')
    count = 1
    for i, name in enumerate(names):
        print('current fit is '+str(count)+' of '+str(len(names)))

        model = fit_types[i]
        parameters = initials[i]
        u = u_vals[i]
        t = tspan
        if name == 'F1T':
            opt = scipy.optimize.minimize(error_func,parameters,args = (model,u,t,yo_vals[i],data[name]), bounds = [[10e3,10e4],[50,500],[0,1],[0,200]])
        else:
            opt = scipy.optimize.minimize(error_func,parameters,args = (model,u,t,yo_vals[i],data[name]))
        fitted_params.append(opt.x)
        residual = opt.fun
        reserror[i] = residual

        print('completed fit '+str(count) + ' with residual sum abs error of '+str(residual))
        count += 1

    fitting_results = {'fit': names, 'type': fit_types, 'initial': initials, 'optimal_parameters': fitted_params,'residuals':[reserror]}
    return fitting_results

In [35]:
def get_initials(types):
    initials = []
    for i, typ in enumerate(types):
        if typ == 'FOPTD':
            initials.append([0.1,100,50])
        elif typ == 'SOPTD':
            initials.append([-0.01,   100,   0.7,
          50])
        elif typ == 'SOZPTD':
            initials.append([-2.83658257e-03,   9.71055133e+01,   6.82392527e-01,
          1.34900330e+02])

    if len(types) == 18:
        initials[-2] = [10e3,100,0.65,50]
    return initials


In [36]:
from Stepping_all import get_results
from scipy.signal import lti
names = ["Ps3Cc_measured"]
import scipy.optimize
types = ['SOPTD']
initials = get_initials(types)
u_vals = [20]
tspan = numpy.linspace(0,2000,1000)
yo_vals = [1.84]
data = get_results()


In [37]:
error_func(initials[0],types[0],u_vals[0],tspan,yo_vals[0],data[names[0]])

125.58436897268355

In [38]:
scipy.optimize.minimize(error_func,initials[0],args = (types[0],u_vals[0],tspan,yo_vals[0],data[names[0]]))

KeyboardInterrupt: 

In [10]:
from Fitting_module import SOPTD
data = SOPTD([-2.83658257e-03,   9.71055133e+01,   6.82392527e-01,
          1.34900330e+02],20,tspan,1.84)

In [11]:
from matplotlib import pyplot as plot
plot.figure()
plot.plot(tspan,data)
plot.show()